In [ ]:
from pathlib import Path

import numpy as np
import torchsummary
from careamics_portfolio import PortfolioManager
from matplotlib.pyplot import imshow, subplots

from careamics_restoration.engine import Engine
from careamics_restoration.metrics import psnr

Import Dataset Portfolio

In [ ]:
# Explore portfolio
portfolio = PortfolioManager()
print(portfolio.denoising)

Read the specific dataset 
bla 

In [ ]:
# Download files
root_path = Path("data")
files = portfolio.denoising.N2V_BSD68.download(root_path)
print(f"List of downloaded files: {files}")

Visualize

Load and visualize config file


In [ ]:
# path_to_config = Path("../src/configs/n2v_2D_config.yml")
# config = config_loader(path_to_config)
# pprint.PrettyPrinter(indent=2).pprint(config)

Initialize the Engine

In [ ]:
engine = Engine(config_path="n2v_2D_BSD.yml")

Visualize tiles ?


Start training

In [ ]:
torchsummary.summary(engine.model, (1, 64, 64))

In [ ]:
engine.train(train_path=Path(files[0]).parent, val_path=Path(files[1]).parent)

In [ ]:
# Temporary hack, because, you know ... una grande cazzata
pred_input = np.load(
    "/home/igor.zubarev/projects/caremics/examples/data/BSD68_reproducibility/BSD68_reproducibility_data/test/img.npy"
)
pred_input_full = np.load(
    "/home/igor.zubarev/projects/caremics/examples/data/BSD68_reproducibility/BSD68_reproducibility_data/test_full/bsd68_gaussian25.npy",
    allow_pickle=True,
)

gt = np.load(
    "/home/igor.zubarev/projects/caremics/examples/data/BSD68_reproducibility/BSD68_reproducibility_data/gt/gt.npy"
)
orig_pred = np.load(
    "/home/igor.zubarev/projects/caremics/examples/data/BSD68_reproducibility/BSD68_reproducibility_data/gt/pred.npy"
)
pred_10ep = np.load(
    "/home/igor.zubarev/projects/caremics/examples/data/BSD68_reproducibility/BSD68_reproducibility_data/gt/pred_tf_10ep.npy"
)
pred_10ep_tf = np.load(
    "/home/igor.zubarev/projects/caremics/examples/data/BSD68_reproducibility/BSD68_reproducibility_data/gt/prediction_tf_n2v_bsd68_gaussian25_0.npy"
)
print(pred_input[0][100:, :156].shape)
imshow(pred_input[0][100:, :156], cmap="gray")

Predict

In [ ]:
preds = engine.predict(
    external_input=pred_input_full[0][np.newaxis, np.newaxis, :100, :156]
)

In [ ]:
print(preds.shape)

In [ ]:
# print(tiles[0][0].shape)

In [ ]:
# preds = tiles[0][0]#np.concatenate(tiles)
print(preds.min(), preds.max())
print(gt.min(), gt.max())
print(orig_pred.min(), orig_pred.max())
print(pred_10ep.min(), pred_10ep.max())
print(pred_10ep_tf.min(), pred_10ep_tf.max())

In [ ]:
imshow(preds.squeeze(), cmap="gray")

Visualize and compute metrics


In [ ]:
print(preds.shape)
_, ax = subplots(2, 2, figsize=(40, 40))
ax[0][0].imshow(pred_input[0], cmap="gray")
ax[0][1].imshow(preds[0].squeeze(), cmap="gray")
ax[1][0].imshow(orig_pred[0], cmap="gray")
ax[1][1].imshow(pred_10ep_tf[:256, :256], cmap="gray")

In [ ]:
print(gt[0].shape, preds.squeeze().shape)
psnr_new = psnr(gt, preds.squeeze())
psnr_old = psnr(gt, orig_pred)

print(f"PSNR new: {psnr_new}, PSNR old: {psnr_old}")

In [ ]:
sub = orig_pred[0] - preds[0].squeeze()
sub_pre = orig_pred[0] - pred_10ep_tf[:256, :256]
sub_n_p = preds[0].squeeze() - pred_10ep_tf[:256, :256]

_, ax = subplots(1, 3, figsize=(40, 40))
ax[0].imshow(sub, cmap="gray")
ax[1].imshow(sub_pre, cmap="gray")
ax[2].imshow(sub_n_p, cmap="gray")